<a href="https://colab.research.google.com/github/Xxwo2/O2O_coupon_predict/blob/master/O2O_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- ref :https://github.com/Xxwo2/O2O-Coupon-Usage-Forecast/blob/master/code/wepon/season%20one/extract_feature.py

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! ls /content/gdrive/My\ Drive/O2O_data

ccf_offline_stage1_test_revised.csv  coupon_feature3.csv    other_feature3.csv
ccf_offline_stage1_train.csv	     merchant_feature3.csv  user_feature3.csv
ccf_online_stage1_train.csv	     O2O_feature.ipynb


In [0]:
import pandas as pd
import numpy as np
from datetime import date

In [0]:
off_train = pd.read_csv('/content/gdrive/My Drive/O2O_data/ccf_offline_stage1_train.csv', header=None)
off_test = pd.read_csv('/content/gdrive/My Drive/O2O_data/ccf_offline_stage1_test_revised.csv', header=None)
on_train = pd.read_csv('/content/gdrive/My Drive/O2O_data/ccf_online_stage1_train.csv', header=None)

In [0]:
off_train.head()

,0,1,2,3,4,5,6
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [0]:
['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']

['user_id',
 'merchant_id',
 'coupon_id',
 'discount_rate',
 'distance',
 'date_received',
 'date']

In [0]:
off_train.columns =['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']
off_test.columns = ['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received']
on_train.columns = ['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']

### data split

- not knowing why split dataset like this

In [0]:
dataset3 = off_test
feature3 = off_train[((off_train.date>=20160315)&(off_train.date<=20160630))|((off_train.date=='null')&(off_train.date_received>=20160315)&(off_train.date_received<=20160630))]
dataset2 = off_train[(off_train.date_received>=20160515)&(off_train.date_received<=20160615)]
feature2 = off_train[((off_train.date>=20160201)&(off_train.date<=20160514))|((off_train.date=='null')&(off_train.date_received>=20160201)&(off_train.date_received<=20160514))]
dataset1 = off_train[(off_train.date_received>=20160414)&(off_train.date_received<=20160514)]
feature1 = off_train[((off_train.date>=20160101)&(off_train.date<=20160413))|((off_train.date=='null')&(off_train.date_received>=20160101)&(off_train.date_received<=20160413))]


/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## for dataset3

### other feature

In [0]:
t = dataset3[['user_id']]
t['this_month_user_receive_all_coupon'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
t = t.groupby('user_id').agg('sum').reset_index()

In [0]:
t1 = dataset3[['user_id', 'coupon_id']]
t1['this_month_user_recieve_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
dataset3.date_received.head()

0    20160712
1    20160706
2    20160727
3    20160727
4    20160708
Name: date_received, dtype: int64

In [0]:
t2 = dataset3[['user_id','coupon_id', 'date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index() #把多个收到日期用":"连起来
t2['receive_number'] = t2.date_received.apply(lambda s : len(s.split(':'))) #把收到的优惠券日期变成列表，并计算列表长度
t2 = t2[t2.receive_number>1] #直接用判断公式做筛选
t2['max_date_received'] = t2.date_received.apply(lambda x: max(x.split(':')))
t2['min_date_received'] = t2.date_received.apply(lambda x: min(x.split(':')))
t2 = t2[['user_id', 'coupon_id', 'max_date_received','min_date_received']]

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### # t3 
计算用户拿到特定优惠券最早和最晚的日期
user received same coupon first time and last time date.

In [0]:
t3 = dataset3[['user_id', 'coupon_id','date_received']]
t3 = pd.merge(t3, t2, on=['user_id', 'coupon_id'], how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received.fillna(0).astype('int64') - t3.date_received.fillna(0).astype('int64')
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.fillna(0).astype('int64') - t3.min_date_received.fillna(0).astype('int64')

In [0]:
def is_firstonelastone(x):
  if x == 0:
    return 1
  elif x >0:
    return 0
  else:
    return -1

In [0]:
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstonelastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstonelastone)

In [0]:
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]


#####  t4

计算用户当天拿到多少总优惠券


In [0]:
t4 = dataset3[['user_id', 'date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
#t4 = 
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### t5

计算用户当天拿到多少相同的优惠券

In [0]:
t5 = dataset3[['user_id', 'coupon_id', 'date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id', 'coupon_id', 'date_received']).agg('sum').reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### t6
把各个优惠券收到的所有日期都放在同一个字段里

In [0]:
t6 = dataset3[['user_id', 'coupon_id', 'date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id', 'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


##### t7

用户收到特定优惠券的时间差

get_day_gap_before = 收到日期 - 每个收到日期
get_day_gap_after = 每个收到日期-收到日期

In [0]:
t7 = dataset3[['user_id', 'coupon_id', 'date_received']]
t7 = pd.merge(t7,t6,on = ['user_id', 'coupon_id'] , how='left')
t7 ['date_received_date'] = t7.date_received.astype('str')+'-'+t7.dates

In [0]:
def get_day_gap_before(s):
  date_received, dates = s.split('-')
  dates = dates.split(':')
  gaps = []
  
  for d in dates:
    this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8])) - date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
    if this_gap >0:
      gaps.append(this_gap)
  
  if len(gaps)==0:
    return -1
  else:
    return min(gaps)
  

In [0]:
def get_day_gap_after(s):
  date_received, dates = s.split('-')
  dates = dates.split(':')
  gaps = []
  
  for d in dates:
    this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
    if this_gap >0:
      gaps.append(this_gap)
  
  if len(gaps)==0:
    return -1
  else:
    return min(gaps)
    

In [0]:
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)

In [0]:
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)

In [0]:
t7 = t7[['user_id', 'coupon_id', 'date_received','day_gap_before', 'day_gap_after']]

##### make them all together

other_feature = 用户总当月总优惠券(t) + 用户当月收到相同优惠券数量(t1)

In [0]:
other_feature3 = pd.merge(t1,t,on = 'user_id') 


In [0]:
other_feature3.head()

,user_id,coupon_id,this_month_user_recieve_same_coupon_count,this_month_user_receive_all_coupon
0,209,825,1,2
1,209,7557,1,2
2,215,5488,1,1
3,316,3992,1,1
4,417,12465,1,1


other_feature = 用户总当月总优惠券(t) + 用户当月收到相同优惠券数量(t1) + 第一次（最后一次）收到优惠券的日期(t3) 

In [0]:
other_feature3 = pd.merge(other_feature3, t3, on = ['user_id','coupon_id'])

In [0]:
other_feature3.head()

,user_id,coupon_id,this_month_user_recieve_same_coupon_count,this_month_user_receive_all_coupon,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,209,825,1,2,20160721,-1,0
1,209,7557,1,2,20160721,-1,0
2,215,5488,1,1,20160703,-1,0
3,316,3992,1,1,20160721,-1,0
4,417,12465,1,1,20160712,-1,0


other_feature = 用户总当月总优惠券(t) + 用户当月收到相同优惠券数量(t1) + 第一次（最后一次）收到优惠券的日期(t3) +用户当天拿到优惠券总数(t4)+用户当天拿到多少相同优惠券(t5)+用户收到特定优惠券的时间差(t7)

In [0]:
other_feature3 = pd.merge(other_feature3, t4, on = ['user_id','date_received'])
other_feature3 = pd.merge(other_feature3, t5, on = ['user_id','coupon_id','date_received'])
other_feature3 = pd.merge(other_feature3, t7, on = ['user_id','coupon_id','date_received'])

In [0]:
other_feature3.shape

(116204, 11)

In [0]:
other_feature3.to_csv('/content/gdrive/My Drive/O2O_data/other_feature3.csv')

In [0]:
other3 = other_feature3

### coupon related

In [0]:
def calc_discount_rate(s):
  s = str(s)
  s = s.split(':')
  if len(s)==1:
    return float(s[0])
  else:
    return 1.0 - float(s[1])/float(s[0])
  
def get_discount_man(s):
  s = str(s)
  s = s.split(':')
  if len(s)==1:
    return 'null'
  else:
    return int(s[0])
  

def get_discount_jian(s):
  s = str(s)
  s = s.split(':')
  if len(s)==1:
    return 'null'
  else:
    return int(s[1])
  
  
def is_man_jian(s):
  s = str(s)
  s = s.split(':')
  if len(s)==1:
    return 0
  else:
    return 1

In [0]:
t = dataset3
t['day_of_week'] = t.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
t['day_of_month'] = t.date_received.astype('str').apply(lambda x:int(x[6:8]))
t['days_distance'] = t.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)
                                                        
                                                   

In [0]:
t['discount_man'] = t.discount_rate.apply(get_discount_man)
t['discount_jian'] = t.discount_rate.apply(get_discount_jian)
t['is_manjian'] = t.discount_rate.apply(is_man_jian)
t['discount_rate'] = t.discount_rate.apply(calc_discount_rate)

In [0]:
t.shape

(113640, 12)

In [0]:
d = dataset3[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby(['coupon_id']).agg('sum').reset_index()


t = pd.merge(t,d,on = 'coupon_id', how = 'left')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
t.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_manjian,coupon_count
0,4129537,450,9983,0.833333,1.0,20160712,2,12,12,30,5,1,11586
1,6949378,1300,3429,0.833333,NaN,20160706,3,6,6,30,5,1,4887
2,2166529,7113,6928,0.900000,5.0,20160727,3,27,27,200,20,1,205
3,2166529,7113,1808,0.900000,5.0,20160727,3,27,27,100,10,1,205
4,6172162,7605,6500,0.966667,2.0,20160708,5,8,8,30,1,1,132


In [0]:
t.to_csv('/content/gdrive/My Drive/O2O_data/coupon_feature3.csv')

In [0]:
coupon3 = t

### merchant related feature

In [0]:
merchant3 = feature3[['merchant_id','coupon_id', 'distance', 'date_received','date']]
t = merchant3[['merchant_id']]
t.drop_duplicates(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


- t1: 消费数量
- t2: 总使用优惠券的消费数量
- t3: 总使用优惠券数量

In [0]:
t1 = merchant3[merchant3.date.isna() == False][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()

t2 = merchant3[(merchant3.date.isna() == False) & (merchant3.coupon_id.isna() == False)][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()

t3 = merchant3[merchant3.coupon_id.isna() == False][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()


t4 = merchant3[(merchant3.date.isna() == False)&(merchant3.coupon_id.isna() == False)][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.fillna(0, inplace = True)
t4.distance = t4.distance.astype('int')
t4.replace(-1, np.nan, inplace = True)

t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns = {'distance': 'merchant_min_distance'}, inplace=True)



t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns = {'distance': 'merchant_max_distance'}, inplace=True)


t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns = {'distance': 'merchant_mean_distance'}, inplace=True)


t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns = {'distance': 'merchant_median_distance'}, inplace=True)




In [0]:
merchant3_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t2,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t3,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t5,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t6,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t7,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t8,on='merchant_id',how='left')
merchant3_feature.sales_use_coupon = merchant3_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_coupon
merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_sales
merchant3_feature.total_coupon = merchant3_feature.total_coupon.replace(np.nan,0) #fillna with 0


新增特征
- merchant_coupon_transfer_rate ：使用coupon的消费数量/总coupon数量
- coupon_rate ： 使用coupon的消费数量/总消费数量

In [0]:
merchant3_feature.to_csv('/content/gdrive/My Drive/O2O_data/merchant_feature3.csv')

In [0]:
merchant3_feature.shape

(8194, 10)

In [0]:
merchant3_feature.head()

,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,2632,8,3.0,3.0,0.0,1.0,0.666667,1.0,1.0,0.375000
1,6459,20,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000
2,1579,44,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,0.022727
3,8390,1726,214.0,214.0,0.0,10.0,0.616822,0.0,1.0,0.123986
4,3051,2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000


In [0]:
merchant3 = merchant3_feature

### user feature

In [0]:
def get_user_date_datereceived_gap(s):
  s = s.split(':')
  try:
    return (date(int(float(s[0][0:4])),int(float(s[0][4:6])),int(float(s[0][6:8]))) -date(int(float(s[1][0:4])),int(float(s[1][4:6])),int(float(s[1][6:8])))).days
  except:
    pass

In [0]:
user3 = feature3[['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']]

t = user3[['user_id']]
t.drop_duplicates(inplace = True)

t1 = user3[user3.date.isna() == False][['user_id', 'merchant_id']]
t1.drop_duplicates(inplace = True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns = {'merchant_id':'count_merchant'}, inplace = True)


t2 = user3[(user3.date.isna() == False)& (user3.coupon_id.isna() == False)][['user_id', 'distance']]
t2.replace('null', -1, inplace=True)
t2.fillna(-1, inplace=True)
t2.distance = t2.distance.astype('int')
t2.replace(-1, np.nan, inplace = True)

t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns = {'distance':'user_min_distance'})

t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns = {'distance':'user_max_distance'})

t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns = {'distance':'user_mean_distance'})

t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns = {'distance':'user_median_distance'})


t7 = user3[(user3.date.isna() == False) & (user3.coupon_id.isna() == False)][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()


t8 = user3[(user3.date.isna() == False)][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()


t9 = user3[user3.coupon_id.isna() == False][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
t10 = user3[(user3.date.isna() == False)&(user3.date_received.isna() == False)][['user_id','date_received','date']]
t10['user_datereceived_gap'] = t10.date.astype('str') + ':' + t10.date_received.astype('str')
t10['user_date_datereceived_gap'] = t10.user_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id', 'user_date_datereceived_gap']]


In [0]:
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)



- buy_use_coupon_rate ：用户所有消费中有使用优惠券的比例
- user_coupon_transfer_rate ：用户所有优惠券的数量中使用优惠券的比例

In [0]:
user3_feature = pd.merge(t,t1,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='user_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.coupon_received.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)


In [0]:
user3_feature.to_csv('/content/gdrive/My Drive/O2O_data/user_feature3.csv')

In [0]:
user3_feature

In [0]:
user3 = user3_feature

##### ignore this

In [0]:
user3 = feature3[['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']]

t = user3[['user_id']]
t.drop_duplicates(inplace = True)

t1 = user3[user3.date != 'null'][['user_id', 'merchant_id']]
t1.drop_duplicates(inplace = True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns = {'merchant_id':'count_merchant'}, inplace = True)


t2 = user3[(user3.date != 'null')& (user3.coupon_id != 'null')][['user_id', 'distance']]
t2.replace('null', -1, inplace=True)
t2.fillna(-1, inplace=True)
t2.distance = t2.distance.astype('int')
t2.replace(-1, np.nan, inplace = True)

t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns = {'distance':'user_min_distance'})

t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns = {'distance':'user_max_distance'})

t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns = {'distance':'user_mean_distance'})

t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns = {'distance':'user_median_distance'})


t7 = user3[(user3.date != 'null') & (user3.coupon_id != 'null')][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()


t8 = user3[(user3.date != 'null')][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()


t9 = user3[user3.coupon_id != 'null'][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()


t10 = user3[(user3.date != 'null')&(user3.date_received != 'null')][['user_id','date_received','date']]
t10['user_datereceived_gap'] = t10.date + ':'+ t10.date_received
t10.user_date_datereceived_gap = t10.uer_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id', 'user_date_datereceived_gap']]

t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns = {'user_date_datereceived_gap':'avg_user_date_datereceived_gap'}, inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns = {'user_date_datereceived_gap':'min_user_date_datereceived_gap'}, inplace=True)
t13 = t10.groupby('user_Id').agg('max').reset_index()
t13.rename(columns = {'user_date_datereceived_gap':'max_user_date_datereceived_gap'}, inplace=True)


user3_feature = pd.merge(t,t1,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='user_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: ignored

### user merchant

In [0]:
all_user_merchant = feature3[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace = True)

t = feature3[['user_id','merchant_id','date']]
t = t[t.date.isna()==False][['user_id', 'merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)


t1 = feature3[['user_id', 'merchant_id', 'coupon_id']]
t1 = t1[t1.coupon_id.isna()==False][['user_id', 'merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)

t2 = feature3[['user_id', 'merchant_id', 'date', 'date_received']]
t2 = t2[(t2.date.isna()==False) & (t2.date_received.isna()==False)][['user_id', 'merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id', 'merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)


t3 = feature3[['user_id', 'merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id', 'merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace = True)


t4 = feature3[['user_id', 'merchant_id', 'date', 'coupon_id']]
t4 = t4[(t4.date.isna()==False)&(t4.coupon_id.isna()==True)][['user_id', 'merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace = True)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


- user_merchant_coupon_transfer_rate : 使用coupon消费的数量/所有收到的coupon数量
- user_merchant_coupon_buy_rate ： 使用coupon消费的数量/所有对商户消费数量
- user_merchant_rate ： 用户对商户的下单次数/用户和商户的互动次数（可能是有消费和只是有优惠券）
- user_merchant_common_buy_rate：不用优惠券购买的比例

In [0]:
user_merchant3 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t1,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t2,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t3,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t4,on=['user_id','merchant_id'],how='left')
user_merchant3.user_merchant_buy_use_coupon = user_merchant3.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant3.user_merchant_buy_common = user_merchant3.user_merchant_buy_common.replace(np.nan,0)
user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')
user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float') / user_merchant3.user_merchant_any.astype('float')
user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')

In [0]:
user_merchant3.to_csv('/content/gdrive/My Drive/O2O_data/user_merchant_feature3.csv')

## 准备数据集

In [0]:
def get_label(s):
    s = s.split(':')
    if s[0]=='null':
        return 0
    elif (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days<=15:
        return 1
    else:
        return -1

In [0]:
dataset3 = pd.merge(coupon3,merchant3,on='merchant_id',how='left')
dataset3 = pd.merge(dataset3,user3,on='user_id',how='left')
dataset3 = pd.merge(dataset3,user_merchant3,on=['user_id','merchant_id'],how='left')
dataset3 = pd.merge(dataset3,other3,on=['user_id','coupon_id','date_received'],how='left')
dataset3.drop_duplicates(inplace=True)

In [0]:
dataset3.shape

(112803, 52)

In [0]:
datasets = dataset3.fillna(0)

In [0]:
datasets.info()

In [0]:
datasets['is_weekend'] = datasets.day_of_week.apply(lambda x: 1 if x in (6,7) else 0)

In [0]:
weekday_dummies = pd.get_dummies(datasets.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]

In [0]:
datasets = pd.concat([datasets, weekday_dummies], axis=1)
datasets.drop(['merchant_id', 'day_of_week','coupon_count'],axis=1, inplace = True)

In [0]:
datasets = datasets.replace('null', 0)

In [0]:
datasets.to_csv('/content/gdrive/My Drive/O2O_data/dataset.csv')

In [0]:
datasets.shape

(112803, 57)

## for dataset2 不知为何join出来没有数据，先做其他的特征处理在dataset3的

same as the what we do for dataset 3


In [0]:
t = dataset2[['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()

t1 = dataset2[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()

t2 = dataset2[['user_id','coupon_id', 'date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index() #把多个收到日期用":"连起来
t2['receive_number'] = t2.date_received.apply(lambda s : len(s.split(':'))) #把收到的优惠券日期变成列表，并计算列表长度
t2 = t2[t2.receive_number>1] #直接用判断公式做筛选
t2['max_date_received'] = t2.date_received.apply(lambda x: max(x.split(':')))
t2['min_date_received'] = t2.date_received.apply(lambda x: min(x.split(':')))
t2 = t2[['user_id', 'coupon_id', 'max_date_received','min_date_received']]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

这边的日期格式要注意，原本是float要换成int或str才能做 - 减法

In [0]:
t3 = dataset3[['user_id', 'coupon_id','date_received']]
t3 = pd.merge(t3, t2, on=['user_id', 'coupon_id'], how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received.fillna(0).astype('float64').astype('int64') - t3.date_received.fillna(0).astype('int64')
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.astype('int64') - t3.min_date_received.fillna(0).astype('float64').astype('int64')

In [0]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once
        
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]


In [0]:
t4 = dataset2[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()

t5 = dataset2[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()

t6 = dataset2[['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)

def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    

t7 = dataset2[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

In [0]:
other_feature2 = pd.merge(t1,t,on='user_id')
other_feature2 = pd.merge(other_feature2,t3,on=['user_id','coupon_id'])
other_feature2 = pd.merge(other_feature2,t4,on=['user_id','date_received'])
#other_feature2 = pd.merge(other_feature2,t5,on=['user_id','coupon_id','date_received'])
#other_feature2 = pd.merge(other_feature2,t7,on=['user_id','coupon_id','date_received'])
#other_feature2.to_csv('data/other_feature2.csv',index=None)


In [0]:
other_feature2.shape

(0, 8)

In [0]:
a1 = pd.merge(t1,t,on='user_id')

In [0]:
pd.merge(a1,t3,on=['user_id','coupon_id']).shape

(3751, 7)